In [1]:
:set -XScopedTypeVariables

import Control.Applicative ((<|>))
import Control.Monad.Trans.State.Strict
import Data.Bool (bool)
import System.Random
import qualified Data.Map.Strict as M
import qualified Data.Set        as S

data Row = R1 | R2 | R3 | R4 | R5 | R6 | R7 | R8 | R9 | R10
    deriving (Bounded, Eq, Ord, Enum, Show)

data Col = Ca | Cb | Cc | Cd | Ce | Cf | Cg | Ch | Ci | Cj
    deriving (Bounded, Eq, Ord, Enum, Show)

data Coord = Coord Row Col
    deriving (Eq, Show, Ord)

allCoords = Coord <$> range <*> range
    where range = toEnum <$> [0..9]

data Ship = Patrol | Cruiser | Submarine | BattleShip | Carrier
    deriving (Eq, Ord, Show, Enum)

data Move = Miss | Hit deriving (Eq, Show)

data Player = Player1 | Player2 deriving (Eq, Show)

type Board = M.Map Ship (S.Set Coord)
type Track = S.Set Coord

data Status = Won Player | InPlay deriving (Show)

data Game = Game 
    { player1Board :: Board
    , player1Track :: Track
    , player2Board :: Board
    , player2Track :: Track
    }
    deriving (Eq, Show)

In [2]:
safeToEnum :: forall a . (Enum a, Bounded a) => Int -> Maybe a
safeToEnum from = if (from < (fromEnum (minBound :: a)) || from > (fromEnum (maxBound :: a))) then Nothing else (Just $ toEnum from)

safeFromTo :: forall a . (Enum a, Bounded a) => Int -> Int -> Maybe [a]
safeFromTo from to = sequence $ map safeToEnum [from..to]

makeHorizontal :: (Int, Int) -> Ship -> Maybe (S.Set Coord)
makeHorizontal (x, y) ship = let
    len    = (fromEnum ship) + 1
    rows   = sequence $ replicate len (safeToEnum y :: Maybe Row)
    cols   = safeFromTo x (x+len) :: Maybe [Col]
    coords = zip <$> rows <*> cols
    in S.fromList . map (uncurry Coord) <$> coords

makeVertical :: (Int, Int) -> Ship -> Maybe (S.Set Coord)
makeVertical (x, y) ship = let
    len    = (fromEnum ship) + 1
    rows   = safeFromTo y (y+len) :: Maybe [Row]
    cols   = sequence $ replicate len (safeToEnum x :: Maybe Col)
    coords = zip <$> rows <*> cols
    in S.fromList . map (uncurry Coord) <$> coords

pair = (,) <$> r <*> r
    where r = randomRIO (0,9)

genBoard' []     board = return board
genBoard' (s:ss) board = do
    p   <- pair
    let cs' = makeHorizontal p s <|> makeVertical p s
    let pr  = S.unions $ M.elems board
    case cs' of
        Just cs | S.intersection cs pr == S.empty -> let
            board' = M.insert s cs board
            in genBoard' ss board'
        _ -> genBoard' (s:ss) board

genBoard = genBoard' [Carrier, BattleShip, Submarine, Patrol, Cruiser] M.empty

In [3]:
makeAttackingMove :: Player -> Coord -> State Game Move
makeAttackingMove Player1 coord = do
    game <- get
    let p2b  = S.unions $ M.elems $ player2Board game
    let p1t  = player1Track game
    let p1t' = S.insert coord p1t
    let move = bool Miss Hit (coord `S.member` p2b)
    put game {player1Track = p1t'}
    return move
makeAttackingMove Player2 coord = do
    game <- get
    let p1b  = S.unions $ M.elems $ player1Board game
    let p2t  = player2Track game
    let p2t' = S.insert coord p2t
    let move = bool Miss Hit (coord `S.member` p1b)
    put game {player2Track = p2t'}
    return move

status :: State Game Status
status = do
    game <- get
    let p1b  = S.unions $ M.elems $ player1Board game
    let p2t  = S.intersection (player2Track game) p1b
    let p2b  = S.unions $ M.elems $ player2Board game
    let p1t  = S.intersection (player2Track game) p2b
    case () of 
        _
            | p1t == p2b -> return $ Won Player1
            | p2t == p1b -> return $ Won Player2
            | otherwise  -> return InPlay

emptyGame = Game M.empty S.empty M.empty S.empty
randomBoard1 <- genBoard
randomBoard2 <- genBoard
board' = runState (makeAttackingMove Player1 (Coord R4 Ci)) emptyGame {player1Board = randomBoard1, player2Board = randomBoard2}
board'
runState status (snd board')

(Miss,Game {player1Board = fromList [(Patrol,fromList [Coord R4 Ci]),(Cruiser,fromList [Coord R6 Ce,Coord R6 Cf]),(Submarine,fromList [Coord R9 Cb,Coord R9 Cc,Coord R9 Cd]),(BattleShip,fromList [Coord R4 Cg,Coord R5 Cg,Coord R6 Cg,Coord R7 Cg]),(Carrier,fromList [Coord R8 Ce,Coord R8 Cf,Coord R8 Cg,Coord R8 Ch,Coord R8 Ci])], player1Track = fromList [Coord R4 Ci], player2Board = fromList [(Patrol,fromList [Coord R4 Ce]),(Cruiser,fromList [Coord R1 Ci,Coord R2 Ci]),(Submarine,fromList [Coord R10 Cg,Coord R10 Ch,Coord R10 Ci]),(BattleShip,fromList [Coord R3 Cj,Coord R4 Cj,Coord R5 Cj,Coord R6 Cj]),(Carrier,fromList [Coord R5 Ca,Coord R5 Cb,Coord R5 Cc,Coord R5 Cd,Coord R5 Ce])], player2Track = fromList []})

(InPlay,Game {player1Board = fromList [(Patrol,fromList [Coord R4 Ci]),(Cruiser,fromList [Coord R6 Ce,Coord R6 Cf]),(Submarine,fromList [Coord R9 Cb,Coord R9 Cc,Coord R9 Cd]),(BattleShip,fromList [Coord R4 Cg,Coord R5 Cg,Coord R6 Cg,Coord R7 Cg]),(Carrier,fromList [Coord R8 Ce,Coord R8 Cf,Coord R8 Cg,Coord R8 Ch,Coord R8 Ci])], player1Track = fromList [Coord R4 Ci], player2Board = fromList [(Patrol,fromList [Coord R4 Ce]),(Cruiser,fromList [Coord R1 Ci,Coord R2 Ci]),(Submarine,fromList [Coord R10 Cg,Coord R10 Ch,Coord R10 Ci]),(BattleShip,fromList [Coord R3 Cj,Coord R4 Cj,Coord R5 Cj,Coord R6 Cj]),(Carrier,fromList [Coord R5 Ca,Coord R5 Cb,Coord R5 Cc,Coord R5 Cd,Coord R5 Ce])], player2Track = fromList []})